In [1]:
import threading

In [45]:
import os
import pickle
ELEICAO_PATH = 'eleicao.p'

#testado
def open_eleicoes():
    '''
    Abre o arquivo com as sessoes já criadas e seus respectivos votos
    '''
    if (os.path.isfile(ELEICAO_PATH)):
        print('open_eleicoes, ja existe eleicao')
        with open(ELEICAO_PATH, 'rb') as fp:
            eleicoes = pickle.load(fp)
    else: eleicoes = []
    return eleicoes

#testado
def save_eleicoes(eleicoes):
    '''
    Salva o resultado da eleicao no arquivo padrão
    '''
    with open(ELEICAO_PATH, 'wb') as fp:
        pickle.dump(eleicoes, fp, protocol=pickle.HIGHEST_PROTOCOL)

#testado
def criarsessao(message):
    '''
    recebe o comando de criar a sessao e a cria de acordo com os parâmetros passados. Assume que já foi checado o comando
    ex de message: criarsessao <nome_sessao> <escolha1> <escolha1>
    '''
    eleicoes = open_eleicoes()
    separate = message.split(' ')

    if (len(separate) < 4):
        return_message = 'Menos de 3 argumentos foram recebidos, por favor refaca a operacao'
    elif (len(separate) > 7):
        return_message = 'Mais de 7 argumentos foram recebidos, por favor refaca a operacao'
    else:
        escolhas = []
        nome_escolhas = []
        for aux in separate[2:]:
            nome_escolhas.append(aux)
        escolhas = [{'id':a, 'nome':nome_escolhas[a], 'votos':[]} for a in range(len(nome_escolhas))]
        # print('escolhas:', escolhas)
        if(len(eleicoes)==0):
            print('len eleicoes == 0')
            eleicao = {'nome':separate[1],'escolhas':escolhas}
            eleicoes = [eleicao]
            return_message = 'Sessao criada com sucesso.'
        else:
            print('len eleicoes != 0')
            def check_name(name, list_dict):
                for a in list_dict:
                    if a['nome'] == name:
                        return 0
                return 1
            # se nao for vazio, checa se ja tem alguma sessao com o mesmo nome
            nome = separate[1]
            if not check_name(nome, eleicoes):
                return_message = 'O nome desta sessao ja existe, tente novamente com um novo nome'
            else:
                eleicoes.append({'nome':separate[1],'escolhas':escolhas})
                save_eleicoes(eleicoes)
                return_message = 'Sessao criada com sucesso.'
    save_eleicoes(eleicoes)
    # print('eleicoes: ', eleicoes)
    return return_message

#testado
def quem_votou(eleicao):
    '''
    recebe uma sessao (dict) e retorna o id dos usuários que já votaram nela
    '''
    result = []
    for escolha in eleicao['escolhas']:
        for voto in escolha['votos']:
            result.append(voto)
    return result

#TESTADO
def executar_voto(mensagem, user_id):
    '''
    com tudo validado, pegaremos a mensagem e efetuaremos o voto
    '''
    eleicoes = open_eleicoes()
    nome = mensagem.split(' ')[1]
    id_escolha = mensagem.split(' ')[2]
    print(f'nome: {nome}, id_escolha: {id_escolha}')
    # Caso o usuário não tenha votado em nenhuma das escolhas de uma dada sessao, o voto será computado
    for eleicao in eleicoes:
        if(eleicao['nome']==nome and user_id not in quem_votou(eleicao)):
            # print('entrou')
            for escolha in eleicao['escolhas']:
                # print()
                if(escolha['id'] == int(id_escolha)): 
                    escolha['votos'].append(user_id)
                    save_eleicoes(eleicoes)
                    return 'Voto computado'
    save_eleicoes(eleicoes)
    return 'Voto não computado, houve algum erro'
                
#TESTADO
def exibir_sessoes():
    '''
    está exibindo todas as sessoes, disponiveis para voto ou não
    '''
    eleicoes = open_eleicoes()
    return str([eleicao['nome'] for eleicao in eleicoes])

#TESTADO
def exibir_escolhas(nome_sessao):
    '''
    exibe as escolhas disponíveis para voto de uma dada sessao
    '''
    sessoes = open_eleicoes()
    for sessao in sessoes:
        if sessao['nome'] == nome_sessao: return str([{'id':escolha['id'], 'nome':escolha['nome']} for escolha in sessao['escolhas']])
    return 'Este nome de sessao é inválido'


#TESTADO
def total_votos_sessao(sessao):
    '''
    Recebe uma sessão (dict) e retorna o total de votos já realizados na mesma.
    '''
    total = 0
    for escolha in sessao['escolhas']:
        total = total + len(escolha['votos'])
    return total


#TESTADO
def exibir_resultados():
    '''
    cada sessao vai ter uma lista de dicts
    '''
    eleicoes = open_eleicoes()
    resultado_eleicoes = []
    for sessao in eleicoes:
        resultado_sessao = []
        nome_sessao = sessao['nome']
        total = total_votos_sessao(sessao)
        if total != 0:
            for escolha in sessao['escolhas']:
                r = {'id':escolha['id'], 'nome':escolha['nome'], 'porcentagem de votos': str(100* len(escolha['votos'] ) / total) + '%'}
                resultado_sessao.append(r)
        else: r = 'Ainda não há votos para essa sessão'

        resultado_eleicoes.append({'Nome da sessao':sessao['nome'], 'resultado':resultado_sessao})    
    return resultado_eleicoes


def app(mensagem, user_id):
    '''
    recebe a mensagem e executa devidamente
    '''
    mensagem_partida = mensagem.split(' ')
    comando = mensagem_partida[0]
    if(comando == 'votar'):
        if(len(mensagem_partida) == 1):
            return exibir_sessoes()
        elif(len(mensagem_partida) == 2):
            nome_sessao = mensagem_partida[1]
            return exibir_escolhas(nome_sessao)
        elif(len(mensagem_partida) == 3):
            return executar_voto(mensagem, user_id)
        else: return 'Parâmetros em excesso'
    elif(comando == 'resultados'):
        return str(exibir_resultados)
    elif(comando == 'criarsessao'):
        return str(criarsessao(mensagem))
    else: return 'error'

# Testando as funções

In [47]:
# total_votos_sessao OK!

sessao = {'nome': 'blos',  'escolhas': [{'id': 0, 'nome': 'bla', 'votos': [2,9,12]}, {'id': 1, 'nome': 'blob', 'votos': [1,3]}]}
# total_votos_sessao(sessao)
# exibir_resultados()
# exibir_sessoes()
# executar_voto(mensagem = 'votar blosssssssssssssss 1', user_id=5)
# quem_votou(sessao)
# criarsessao('criarsessao blasbstersagraaaado blabster sagrado aq')
app(mensagem='criarsessao sessaob opcao1 opcao2', user_id=3)

open_eleicoes, ja existe eleicao
len eleicoes != 0


'Sessao criada com sucesso.'

In [26]:
# exibir_resultados()
# exibir_resultados()

In [27]:

exibir_escolhas('blos')

open_eleicoes, ja existe eleicao


"[{'id': 0, 'nome': 'bla'}, {'id': 1, 'nome': 'blob'}]"

In [9]:
teste = [{'id': 0, 'nome': 'bla', 'votos': []}, {'id': 1, 'nome': 'blob', 'votos': [1]}]


In [57]:
from ZVprotect import ZVprotect as ZV

In [63]:
# 0 chave publica, privada, primos multiplicados, e o objeto
publickey, privatekey, n, _ = ZV.CAssimetrica.gerarChaves()


Escolha sua chave pública:

[3, 5, 7, 9, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 57, 59, 61, 63, 67, 69, 71, 73, 75, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 101, 103, 105, 107, 109, 113, 115, 117, 119, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 167, 169, 171, 173, 177, 179, 181, 183, 185, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 233, 235, 237, 239, 243, 245, 247, 249, 251, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 277, 279, 281, 283, 285, 289, 291, 293, 295, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 321, 323, 325, 327, 329, 331, 333, 335, 337, 339, 343, 345, 347, 349]



In [67]:
for a in range(2,4):
        publickey, privatekey, n, _ = ZV.CAssimetrica.gerarChaves()
        with open(f'client{a}_public_key.key', 'wb') as fp:
                pickle.dump(publickey, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open(f'client{a}_private_key.key', 'wb') as fp:
                pickle.dump(privatekey, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open(f'client{a}n.key', 'wb') as fp:
                pickle.dump(n, fp, protocol=pickle.HIGHEST_PROTOCOL)

Escolha sua chave pública:

[7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 49, 53, 59, 61, 67, 71, 73, 77, 79, 83, 89, 91, 97, 101, 103, 107, 113, 119, 121, 127, 131, 133, 137, 139, 143, 149, 151, 157, 163, 167, 169, 173, 179, 181, 187, 191, 193, 197, 199, 203, 209, 211, 217, 221, 223, 227, 229, 233, 239, 241, 247, 251, 253, 257, 259, 263, 271, 277, 281, 283, 287, 289, 293, 299, 301, 307, 311, 313, 317, 319, 323, 329, 331, 337, 341, 343, 347, 349, 353, 359, 361, 367, 371, 373, 377, 383, 389, 391, 397, 401, 403, 407, 409, 413, 419, 421, 427, 433, 437, 439, 443, 449, 451, 457, 461, 463, 467, 469, 473, 479, 481, 487, 491, 493, 497, 499, 503, 509, 511, 517, 521, 523, 527, 529, 533]

Escolha sua chave pública:

[3, 5, 7, 9, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 57, 59, 61, 63, 67, 69, 71, 73, 75, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 101, 103, 105, 107, 109, 113, 115, 117, 119, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 145, 147, 149, 151,

In [53]:
serverSocket = ZVprotect.Conexao.conectar(2)

DESKTOP-H50GRPF


In [ ]:
    
PRIVADA_PATH = 'path'

def salvar_chave_privada():
    '''

    '''
    if (os.path.isfile(PRIVADA_PATH)):
        print('chave privada ja existe')
        with open(ELEICAO_PATH, 'rb') as fp:
            eleicoes = pickle.load(fp)
    else: eleicoes = []
    return eleicoes

def save_chave(chave):
    '''
    Salva o resultado da eleicao no arquivo padrão
    '''
    with open(PRIVADA_PATH, 'wb') as fp:
        pickle.dump(chave, fp, protocol=pickle.HIGHEST_PROTOCOL)